In [155]:
import random
import torch
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt
from torch import nn ,optim, Tensor
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import pandas as pd
from torch_geometric.utils import structured_negative_sampling, degree
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [156]:
movie_path = '../data/ml-latest-small/movies.csv'
rating_path = '../data/ml-latest-small/ratings.csv'
print(pd.read_csv(movie_path).head())
print(pd.read_csv(rating_path).head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [157]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [158]:
from sentence_transformers import SentenceTransformer
class SequenceEncoder:
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [159]:
class GenresEncoder:
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

In [160]:
movie_x, movie_mapping = load_node_csv(
    movie_path, index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder()
    })

Batches: 100%|██████████| 305/305 [00:03<00:00, 92.80it/s] 


In [161]:
user_x, user_mapping = load_node_csv(rating_path, index_col='userId')

In [162]:
from torch_geometric.data import HeteroData
data = HeteroData()
data['users'].num_nodes = len(user_mapping)  # Users do not have any features.
data['items'].x = movie_x

In [163]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [164]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [165]:
edge_index, edge_label = load_edge_csv(
    rating_path,
    src_index_col='movieId',
    src_mapping=movie_mapping,
    dst_index_col='userId',
    dst_mapping=user_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

data['user', 'rates', 'items'].edge_index = edge_index
data['user', 'rates', 'items'].edge_label = edge_label
print(data)

HeteroData(
  users={ num_nodes=610 },
  items={ x=[9742, 404] },
  (user, rates, items)={
    edge_index=[2, 100836],
    edge_label=[100836, 1],
  }
)


In [166]:
num_node = len(user_mapping)+len(movie_mapping)
user_emb = nn.Embedding(num_embeddings=len(user_mapping)+1, embedding_dim=movie_x.size(dim=1))
nn.init.normal_(user_emb.weight, std=0.1)
user_x=torch.arange(1,len(user_mapping)+1).view(1,-1)
user_x = torch.LongTensor(user_x)
user_x = user_emb(user_x)
user_x = user_x.squeeze(0)
data['users'].x = user_x

In [167]:
def sample_mini_batch(batch_size, edge_index):
    """
    Args:
        batch_size (int): 批大小
        edge_index (torch.Tensor): 2*N的边列表
    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

In [168]:
class GNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GNN, self).__init__(aggr='mean', flow='source_to_target')
        self.linear = torch.nn.Linear(in_channels, out_channels)
        self.relu = torch.nn.ReLU()

    def forward(self, data: HeteroData):
        x_src, x_tgt, edge_index = data['items'].x, data['users'].x, data['user', 'rates', 'items'].edge_index
        out = self.propagate(x_src=x_src, x_tgt=x_tgt,edge_index=edge_index)
        out = self.linear(out)
        out = self.relu(out)
        return out
    
    def message(self, x_src,edge_index):
        message_src = x_src[edge_index[0]]
        return message_src
    
    def aggregate(self, inputs, index):
        # inputs 是消息，index 指定目标节点的索引
        print("inputs.size() = ",inputs.size())
        print("index.size() = ",index.size())
        return super().aggregate(inputs, index)
    
    def update(self, aggr_out):
        print("aggr_out.size() = ",aggr_out.size())
        return aggr_out
    

In [169]:
model = GNN(user_x.size(dim=1),user_x.size(dim=1))
model = model.to(device)
data = data.to(device)


In [170]:

emb = model.forward(data)
print(emb.size())

inputs.size() =  torch.Size([100836, 404])
index.size() =  torch.Size([100836])
aggr_out.size() =  torch.Size([610, 404])
torch.Size([610, 404])


In [171]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """
    Args:
        users_emb_final (torch.Tensor): e_u^k
        users_emb_0 (torch.Tensor): e_u^0
        pos_items_emb_final (torch.Tensor): positive e_i^k
        pos_items_emb_0 (torch.Tensor): positive e_i^0
        neg_items_emb_final (torch.Tensor): negative e_i^k
        neg_items_emb_0 (torch.Tensor): negative e_i^0
        lambda_val (float): λ的值
    Returns:
        torch.Tensor: loss值
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2))  # L2 loss L2范数是指向量各元素的平方和然后求平方根

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1) # 正采样预测分数
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1) # 负采样预测分数

    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

In [172]:
# model.train()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# for iter in iteration:
#     users_emb0 , items_emb0 ,users_embk = model.forward(data,edge_index)
#     loss = bpr_loss(users_embk, items_emb0)
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()
#     if iter % 10 == 0:
#         loss= loss
#         print("loss = ",loss)
# print("Train done !")


In [173]:

# def lossfunction(users_embk, items_emb0,edge_index):
#     users_embk = users_embk[edge_index[1]]
#     items_emb0 = items_emb0[edge_index[0]]
#     items_emb0 = items_emb0.t()
#     simliarity = torch.matmul(users_embk , items_emb0)
#     simliarity = torch.sum(simliarity, dim=0)
#     loss = torch.mean(simliarity)
#     return 0


In [174]:

def lossfunction(users_embk, items_emb0, edge_index, batch_size=1024):
    """使用批处理计算损失，减少内存使用"""
    total_loss = 0
    num_edges = edge_index.size(1)
    num_batches = (num_edges + batch_size - 1) // batch_size
    
    for i in range(num_batches):
        # 获取当前批次的边索引
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_edges)
        batch_edge_index = edge_index[:, start_idx:end_idx]
        
        # 获取当前批次的用户和物品嵌入
        batch_users = users_embk[batch_edge_index[1]]
        batch_items = items_emb0[batch_edge_index[0]]
        
        # 计算相似度
        simliarity = torch.sum(batch_users * batch_items, dim=1)  # 使用元素乘法代替矩阵乘法
        batch_loss = torch.mean(simliarity)
        total_loss += batch_loss
    
    return total_loss / num_batches

In [98]:
learning_rate = 1e-1
batch_size = 64
iteration = 5000
lambda_val = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
losses = []

  0%|          | 2/5000 [00:00<06:10, 13.48it/s]

loss =  tensor(0.0638, device='cuda:0', grad_fn=<DivBackward0>)


  2%|▏         | 104/5000 [00:06<05:16, 15.48it/s]

loss =  tensor(-364.8804, device='cuda:0', grad_fn=<DivBackward0>)


  4%|▍         | 204/5000 [00:13<05:12, 15.34it/s]

...

loss =  tensor(-1459.5350, device='cuda:0', grad_fn=<DivBackward0>)


 10%|█         | 504/5000 [00:32<04:54, 15.26it/s]

loss =  tensor(-1824.2296, device='cuda:0', grad_fn=<DivBackward0>)


 10%|█         | 509/5000 [00:33<04:53, 15.30it/s]


KeyboardInterrupt: 

In [ ]:
for iter in trange(iteration):
    users_emb = model.forward(data)
    #edges = structured_negative_sampling(data['user', 'rates', 'items'].edge_index)
    #edges = torch.stack(edges, dim=0)
    loss = lossfunction(users_emb,data['items'].x,data['user', 'rates', 'items'].edge_index)
    # 记录损失
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if iter % 100 == 0:
        print("loss = ",loss)
print("Train done !")


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('Training Loss Over Time')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid(True)
plt.show()
